In [ ]:
# general imports

import os
import sys
import itertools
sys.path.append(os.path.expanduser('~/PythonModules/Own'))

import seaborn as sns
import numpy as np
import scipy.signal as sig
import scipy.constants as csts
import scipy.odr as odr
import matplotlib.pyplot as plt
#import master_equation_simulation_py3 as meq # no python3 support :-((((

from mpl_toolkits.axes_grid1 import make_axes_locatable

# importing own packages and methods

import fitting_functions as func

from plotting import *

# notebook setup

set_sns_standard(context='talk')

# do some magix
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# import list with residue names
residue_names = []
with open('../working_files/residues_WW.txt') as f:
    for line in f:
        index, name = line.split()
        residue_names.append(index+' '+name)
        
print(residue_names)

In [ ]:
def plot_E_kin(E_kin_array, time, plotlabels, saveroot='E_kin_MD', E_kin_model=None, t_model=None, log=True, norm_to_zero=True):
    fig = plt.figure(figsize=(10,6))
    axe = fig.add_subplot(111)
    
    for i, e_kin in enumerate(E_kin_array):
        if norm_to_zero:
            lineplot = axe.plot(time, e_kin-e_kin[0], label=plotlabels[i])
        else:
            lineplot = axe.plot(time, e_kin, label=plotlabels[i])

        if type(E_kin_model) != type(None):
            if norm_to_zero:
                axe.plot(t_model, E_kin_model[i]-E_kin_model[i,0], '--', color = lineplot[0].get_color())
            else:
                axe.plot(t_model, E_kin_model[i], '--', color = lineplot[0].get_color())
    
    if log:
        axe.set_xscale('log')
    axe.legend()
    axe.set_xlabel('t / ps')
    axe.set_ylabel('E$_{kin}$ / kJ mol$^{-1}$')
    
    if type(E_kin_model) == type(None): 
        axe.set_xlim(xmin=min(time), xmax=max(time))
    elif type(E_kin_model) == type(np.zeros(0)):
        axe.set_xlim(xmin=min(t_model), xmax=max(t_model))

    
    savename = '/home/thiele/Documents/Physik/HiWi/WW/Plots/' + saveroot
    for resname in plotlabels:
        savename += '_{0}'.format(resname.replace(' ', ''))
    if not log:
        savename += '_lin'
    savename += '.pdf'
    
    fig.tight_layout()
    fig.savefig(savename)

# MD Data

## Raw results

In [ ]:
E_kin_MD = np.load('../working_files/E_kin_all_residues_average.npy')/100 # divide by 100 due to MDanalysis units
t = np.load('../working_files/time_scale.npy')
time_MD = t

# apply savitzki golay filter to data
'''exponential_intervals = [0]
exponential_intervals += [10**i for i in range(1, int(round(np.log10(len(E_kin_MD[0])))))]

for ekin in E_kin_MD:
    for i in range(len(exponential_intervals)-1):
        if i < len(exponential_intervals)-2:
            index1 = exponential_intervals[i]
            index2 = exponential_intervals[i+1]
            ekin_unfiltered = ekin[index1:index2]
            ekin_filtered = sig.savgol_filter(ekin_unfiltered, 5**(i+1), 4)
            ekin[index1:index2] = ekin_filtered
            
        elif i == len(exponential_intervals)-2:
            index1 = exponential_intervals[i]
            ekin_unfiltered = ekin[index1:]
            ekin_filtered = sig.savgol_filter(ekin_unfiltered, 5**(i+1), 4)
            ekin[index1:] = ekin_filtered'''


plotlabels = residue_names[1:]

    
plot_E_kin(E_kin_MD[12:17], t, plotlabels[12:17])

plot_E_kin(E_kin_MD[17:18], t, plotlabels[17:18], norm_to_zero=False)

plot_E_kin(E_kin_MD[18:23], t, plotlabels[18:23])

plot_E_kin(E_kin_MD[23:26], t, plotlabels[23:26])


plot_E_kin(E_kin_MD[5:8], t, plotlabels[5:8])

plot_E_kin(E_kin_MD[26:], t, plotlabels[26:])

plot_E_kin(E_kin_MD[17:18, :1000], t[:1000], plotlabels[17:18], log=False)
plot_E_kin(E_kin_MD[15:17], t, plotlabels[15:17], log=False)


# total energy of peptide
plot_E_kin([np.sum(E_kin_MD, axis=0)[:1000]], time_MD[:1000], plotlabels=['Total energy'], saveroot='E_kin_tot_MD')

## Exponential fit of the decay rate of the heater

In [ ]:
# fit the decay rate of azu to the data
def exp_func(B, x):
    return func.ExponentialC(x, *B)

def f(B, x):
    '''Linear function y = m*x + b'''
    return B[0]*x + B[1]

#exponential fit
exp_model = odr.Model(exp_func)
print(len(t), len(E_kin_MD[17:18][0]), t[10], t[2000])
azu_data = odr.Data(t[10:2000], E_kin_MD[17:18, 10:2000][0])
azu_odr = odr.ODR(azu_data, exp_model, beta0=[160, -10, 0.21, 0])

azu_fit_results = azu_odr.run()

azu_fit_results.pprint()
popt = azu_fit_results.beta
print(popt)

fig =  plt.figure()
axe = fig.add_subplot(111)

axe.plot(t[10:2000], E_kin_MD[17:18, 10:2000][0])
axe.plot(t, exp_func(popt,t))
axe.set_xscale('log')

#linear fit of logged data
lin_model = odr.Model(f)
#azu_data_ln = 

# Comparison of MD and MEQ

In [ ]:
E_kin_MEQ = np.load('../working_files/E_kin_all_residues_MEQ.npy')/100

plot_E_kin(E_kin_MD[15:17], time_MD, plotlabels[15:17], saveroot='E_kin_MEQ_MD', E_kin_model=E_kin_MEQ[16:18], t_model=time_MEQ)
plot_E_kin(E_kin_MD[17:18], time_MD, plotlabels[17:18], saveroot='E_kin_MEQ_MD', E_kin_model=E_kin_MEQ[18:19], t_model=time_MEQ)
plot_E_kin(E_kin_MD[18:23], time_MD, plotlabels[18:23], saveroot='E_kin_MEQ_MD', E_kin_model=E_kin_MEQ[19:24], t_model=time_MEQ)
plot_E_kin(E_kin_MD[6:9], time_MD, plotlabels[6:9], saveroot='E_kin_MEQ_MD', E_kin_model=E_kin_MEQ[7:10], t_model=time_MEQ)
plot_E_kin(E_kin_MD[26:], time_MD, plotlabels[26:], saveroot='E_kin_MEQ_MD', E_kin_model=E_kin_MEQ[27:29], t_model=time_MEQ)


In [ ]:
polar_rates_matrix = np.load('../working_files/polar_rates_matrix.npy')
axis = np.arange(1,37, 5)

def plot_matrix(matrix, xticklabels, yticklabels):
    fig = plt.figure(figsize=(10,10))
    axe = fig.add_subplot(111, aspect=1.0)

    mappable = axe.imshow(matrix, extent = [min(xticklabels), max(xticklabels), max(yticklabels), min(yticklabels)], cmap = sns.cubehelix_palette(light=1, as_cmap=True), interpolation='none')
    
    axe.xaxis.tick_top()
    axe.yaxis.tick_left()
    
    divider = make_axes_locatable(axe)
    cax = divider.append_axes("right", size="5%", pad=0.5)

    plt.colorbar(mappable, cax=cax)
    
    return fig
    
    
print(polar_rates_matrix[8,17], polar_rates_matrix[8,18], polar_rates_matrix[8,19])
print(polar_rates_matrix[27,:])
    
fig = plot_matrix(polar_rates_matrix, xticklabels=axis, yticklabels=axis)
fig.savefig('../Plots/polar_rates_matrix.png')

In [ ]:
bonded_rates_matrix = np.load('../working_files/bonded_rates_matrix.npy')
axis = np.arange(1,35)

polar_rates_matrix = np.load('../working_files/polar_rates_matrix.npy')
axis = np.arange(1,36, 5)
  
fig = plot_matrix(bonded_rates_matrix, xticklabels=axis, yticklabels=axis)
fig.savefig('../Plots/bonded_rates_matrix.pdf')

bonded_rates_matrix[27, 28]

In [ ]:
k_B = csts.Boltzmann
mol = csts.physical_constants['Avogadro constant'][0]

ekin = 60 * 67/2*k_B*910*mol/1000
print(ekin)

# approximate temperature of valine at begining.
180/k_B/mol*1000*2/39/19